In [1]:
from couplets_utils import *
from keras.layers import *
from keras.models import Model
from keras import backend as K
import numpy as np
import tensorflow as tf

/Users/zhangshulin_work/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
N_A = 128

In [3]:
sample = load_sample_datasets(vocabs_size=2000, max_len=30, batch_size=16, sample_size=100, n_a=N_A)
sample_gen = sample['sample_gen']
index2word = sample['index2word']
word2index = sample['word2index']
vocabs_size = sample['vocabs_size']
max_len = sample['max_len']

In [4]:
print('sample_gen length :', len(sample_gen))

sample_gen length : 7


In [20]:
def train_model(n_x, n_a, Tx):
    input = Input(shape=(Tx, n_x), name='x0')
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
        
    a = a0
    c = c0
    
    lstm_cell = LSTM(units=n_a, return_state=True)
    dense_layer = Dense(units=n_x, activation='softmax')
        
    outputs = []
        
    for i in range(Tx):
        x = Lambda(lambda j: j[:, i, :])(input)
        x = Reshape(target_shape=(1, -1))(x)
        a, x, c = lstm_cell(x, initial_state=[a, c])
        x = dense_layer(a)
        outputs.append(x)
                            
    model = Model(inputs=[input, a0, c0], outputs=outputs)
        
    return model

In [21]:
train_model = train_model(vocabs_size, N_A, max_len)

In [22]:
train_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
train_model.fit_generator(sample_gen, epochs=100)

Epoch 1/100
7/7 [==============================] - 1s 156ms/step - loss: 116.8247 - dense_5_loss_1: 7.5006 - dense_5_loss_2: 7.4261 - dense_5_loss_3: 7.2784 - dense_5_loss_4: 7.0398 - dense_5_loss_5: 6.4666 - dense_5_loss_6: 5.8209 - dense_5_loss_7: 6.6106 - dense_5_loss_8: 4.6850 - dense_5_loss_9: 6.5421 - dense_5_loss_10: 6.3463 - dense_5_loss_11: 6.4065 - dense_5_loss_12: 5.7347 - dense_5_loss_13: 5.0439 - dense_5_loss_14: 5.1390 - dense_5_loss_15: 5.1217 - dense_5_loss_16: 3.3962 - dense_5_loss_17: 1.9906 - dense_5_loss_18: 1.8678 - dense_5_loss_19: 2.1333 - dense_5_loss_20: 1.8638 - dense_5_loss_21: 1.7527 - dense_5_loss_22: 1.7491 - dense_5_loss_23: 1.7552 - dense_5_loss_24: 1.6508 - dense_5_loss_25: 1.5609 - dense_5_loss_26: 1.1569 - dense_5_loss_27: 0.8404 - dense_5_loss_28: 0.7082 - dense_5_loss_29: 0.6366 - dense_5_loss_30: 0.6001 - dense_5_acc_1: 0.0000e+00 - dense_5_acc_2: 0.0000e+00 - dense_5_acc_3: 0.0000e+00 - dense_5_acc_4: 0.0000e+00 - dense_5_acc_5: 0.0000e+00 - dense

7/7 [==============================] - 1s 153ms/step - loss: 115.3077 - dense_5_loss_1: 7.4704 - dense_5_loss_2: 7.3684 - dense_5_loss_3: 7.1753 - dense_5_loss_4: 6.8862 - dense_5_loss_5: 6.2708 - dense_5_loss_6: 5.6700 - dense_5_loss_7: 6.4372 - dense_5_loss_8: 4.5670 - dense_5_loss_9: 6.4975 - dense_5_loss_10: 6.3209 - dense_5_loss_11: 6.3835 - dense_5_loss_12: 5.7289 - dense_5_loss_13: 5.0543 - dense_5_loss_14: 5.1393 - dense_5_loss_15: 5.1036 - dense_5_loss_16: 3.3726 - dense_5_loss_17: 1.9705 - dense_5_loss_18: 1.8377 - dense_5_loss_19: 2.1146 - dense_5_loss_20: 1.8394 - dense_5_loss_21: 1.7309 - dense_5_loss_22: 1.7243 - dense_5_loss_23: 1.7292 - dense_5_loss_24: 1.6281 - dense_5_loss_25: 1.5296 - dense_5_loss_26: 1.1257 - dense_5_loss_27: 0.8056 - dense_5_loss_28: 0.6695 - dense_5_loss_29: 0.5975 - dense_5_loss_30: 0.5589 - dense_5_acc_1: 0.0000e+00 - dense_5_acc_2: 0.0000e+00 - dense_5_acc_3: 0.0000e+00 - dense_5_acc_4: 0.0000e+00 - dense_5_acc_5: 0.0000e+00 - dense_5_acc_6: 0.

7/7 [==============================] - 1s 153ms/step - loss: 111.2803 - dense_5_loss_1: 7.4371 - dense_5_loss_2: 7.3002 - dense_5_loss_3: 7.0522 - dense_5_loss_4: 6.7316 - dense_5_loss_5: 6.1126 - dense_5_loss_6: 5.5129 - dense_5_loss_7: 6.2326 - dense_5_loss_8: 4.1602 - dense_5_loss_9: 6.3075 - dense_5_loss_10: 6.1115 - dense_5_loss_11: 6.2093 - dense_5_loss_12: 5.5587 - dense_5_loss_13: 4.8815 - dense_5_loss_14: 4.9999 - dense_5_loss_15: 4.9860 - dense_5_loss_16: 3.2805 - dense_5_loss_17: 1.9211 - dense_5_loss_18: 1.7680 - dense_5_loss_19: 2.0354 - dense_5_loss_20: 1.7424 - dense_5_loss_21: 1.6309 - dense_5_loss_22: 1.6229 - dense_5_loss_23: 1.6173 - dense_5_loss_24: 1.5163 - dense_5_loss_25: 1.4178 - dense_5_loss_26: 1.0122 - dense_5_loss_27: 0.6865 - dense_5_loss_28: 0.5465 - dense_5_loss_29: 0.4655 - dense_5_loss_30: 0.4234 - dense_5_acc_1: 0.0000e+00 - dense_5_acc_2: 0.0000e+00 - dense_5_acc_3: 0.0000e+00 - dense_5_acc_4: 0.0000e+00 - dense_5_acc_5: 0.0000e+00 - dense_5_acc_6: 0.

7/7 [==============================] - 1s 154ms/step - loss: 106.4732 - dense_5_loss_1: 7.3982 - dense_5_loss_2: 7.2183 - dense_5_loss_3: 6.9094 - dense_5_loss_4: 6.5753 - dense_5_loss_5: 5.9565 - dense_5_loss_6: 5.4031 - dense_5_loss_7: 6.0823 - dense_5_loss_8: 3.9431 - dense_5_loss_9: 6.0768 - dense_5_loss_10: 5.8708 - dense_5_loss_11: 5.9859 - dense_5_loss_12: 5.3366 - dense_5_loss_13: 4.6863 - dense_5_loss_14: 4.8125 - dense_5_loss_15: 4.8373 - dense_5_loss_16: 3.1110 - dense_5_loss_17: 1.8019 - dense_5_loss_18: 1.6191 - dense_5_loss_19: 1.8712 - dense_5_loss_20: 1.5769 - dense_5_loss_21: 1.4790 - dense_5_loss_22: 1.4689 - dense_5_loss_23: 1.4683 - dense_5_loss_24: 1.3607 - dense_5_loss_25: 1.2584 - dense_5_loss_26: 0.8454 - dense_5_loss_27: 0.5439 - dense_5_loss_28: 0.3948 - dense_5_loss_29: 0.3158 - dense_5_loss_30: 0.2656 - dense_5_acc_1: 0.0000e+00 - dense_5_acc_2: 0.0000e+00 - dense_5_acc_3: 0.0000e+00 - dense_5_acc_4: 0.0000e+00 - dense_5_acc_5: 0.0000e+00 - dense_5_acc_6: 0.

7/7 [==============================] - 1s 154ms/step - loss: 102.8654 - dense_5_loss_1: 7.3584 - dense_5_loss_2: 7.1300 - dense_5_loss_3: 6.7646 - dense_5_loss_4: 6.4290 - dense_5_loss_5: 5.8137 - dense_5_loss_6: 5.3226 - dense_5_loss_7: 6.0127 - dense_5_loss_8: 3.8457 - dense_5_loss_9: 5.9473 - dense_5_loss_10: 5.7205 - dense_5_loss_11: 5.8326 - dense_5_loss_12: 5.1421 - dense_5_loss_13: 4.5575 - dense_5_loss_14: 4.6667 - dense_5_loss_15: 4.6795 - dense_5_loss_16: 2.8834 - dense_5_loss_17: 1.7311 - dense_5_loss_18: 1.5194 - dense_5_loss_19: 1.7592 - dense_5_loss_20: 1.4476 - dense_5_loss_21: 1.3281 - dense_5_loss_22: 1.3216 - dense_5_loss_23: 1.3338 - dense_5_loss_24: 1.2507 - dense_5_loss_25: 1.1781 - dense_5_loss_26: 0.7601 - dense_5_loss_27: 0.4531 - dense_5_loss_28: 0.3046 - dense_5_loss_29: 0.2111 - dense_5_loss_30: 0.1607 - dense_5_acc_1: 0.0000e+00 - dense_5_acc_2: 0.0000e+00 - dense_5_acc_3: 0.0091 - dense_5_acc_4: 0.0000e+00 - dense_5_acc_5: 0.0000e+00 - dense_5_acc_6: 0.1141

...

7/7 [==============================] - 1s 150ms/step - loss: 85.0564 - dense_5_loss_1: 6.8136 - dense_5_loss_2: 6.0590 - dense_5_loss_3: 5.5192 - dense_5_loss_4: 5.3208 - dense_5_loss_5: 4.7254 - dense_5_loss_6: 4.5783 - dense_5_loss_7: 5.3623 - dense_5_loss_8: 3.0397 - dense_5_loss_9: 5.3680 - dense_5_loss_10: 5.0554 - dense_5_loss_11: 5.1573 - dense_5_loss_12: 4.2498 - dense_5_loss_13: 3.8480 - dense_5_loss_14: 3.9546 - dense_5_loss_15: 3.9471 - dense_5_loss_16: 1.6893 - dense_5_loss_17: 1.3264 - dense_5_loss_18: 1.0354 - dense_5_loss_19: 1.3366 - dense_5_loss_20: 1.0371 - dense_5_loss_21: 1.0147 - dense_5_loss_22: 1.0055 - dense_5_loss_23: 1.0252 - dense_5_loss_24: 0.8910 - dense_5_loss_25: 0.8394 - dense_5_loss_26: 0.3638 - dense_5_loss_27: 0.2896 - dense_5_loss_28: 0.0903 - dense_5_loss_29: 0.0808 - dense_5_loss_30: 0.0326 - dense_5_acc_1: 0.0272 - dense_5_acc_2: 0.0453 - dense_5_acc_3: 0.0000e+00 - dense_5_acc_4: 0.0000e+00 - dense_5_acc_5: 0.1812 - dense_5_acc_6: 0.1956 - dense_

7/7 [==============================] - 1s 156ms/step - loss: 83.5117 - dense_5_loss_1: 6.7572 - dense_5_loss_2: 5.9699 - dense_5_loss_3: 5.4394 - dense_5_loss_4: 5.2449 - dense_5_loss_5: 4.6065 - dense_5_loss_6: 4.4501 - dense_5_loss_7: 5.2812 - dense_5_loss_8: 2.9225 - dense_5_loss_9: 5.3102 - dense_5_loss_10: 4.9851 - dense_5_loss_11: 5.0799 - dense_5_loss_12: 4.1773 - dense_5_loss_13: 3.7869 - dense_5_loss_14: 3.8976 - dense_5_loss_15: 3.8829 - dense_5_loss_16: 1.6254 - dense_5_loss_17: 1.2964 - dense_5_loss_18: 1.0057 - dense_5_loss_19: 1.3144 - dense_5_loss_20: 1.0106 - dense_5_loss_21: 0.9950 - dense_5_loss_22: 0.9860 - dense_5_loss_23: 1.0042 - dense_5_loss_24: 0.8705 - dense_5_loss_25: 0.8286 - dense_5_loss_26: 0.3334 - dense_5_loss_27: 0.2688 - dense_5_loss_28: 0.0791 - dense_5_loss_29: 0.0748 - dense_5_loss_30: 0.0273 - dense_5_acc_1: 0.0272 - dense_5_acc_2: 0.0869 - dense_5_acc_3: 0.0000e+00 - dense_5_acc_4: 0.0000e+00 - dense_5_acc_5: 0.1812 - dense_5_acc_6: 0.1956 - dense_

7/7 [==============================] - 1s 155ms/step - loss: 82.2327 - dense_5_loss_1: 6.6999 - dense_5_loss_2: 5.8823 - dense_5_loss_3: 5.3620 - dense_5_loss_4: 5.1722 - dense_5_loss_5: 4.5011 - dense_5_loss_6: 4.3312 - dense_5_loss_7: 5.1933 - dense_5_loss_8: 2.8182 - dense_5_loss_9: 5.2425 - dense_5_loss_10: 4.9142 - dense_5_loss_11: 5.0269 - dense_5_loss_12: 4.1225 - dense_5_loss_13: 3.7296 - dense_5_loss_14: 3.8524 - dense_5_loss_15: 3.8645 - dense_5_loss_16: 1.5909 - dense_5_loss_17: 1.2742 - dense_5_loss_18: 0.9846 - dense_5_loss_19: 1.2998 - dense_5_loss_20: 0.9926 - dense_5_loss_21: 0.9789 - dense_5_loss_22: 0.9722 - dense_5_loss_23: 0.9910 - dense_5_loss_24: 0.8534 - dense_5_loss_25: 0.8103 - dense_5_loss_26: 0.3319 - dense_5_loss_27: 0.2664 - dense_5_loss_28: 0.0779 - dense_5_loss_29: 0.0731 - dense_5_loss_30: 0.0227 - dense_5_acc_1: 0.0272 - dense_5_acc_2: 0.0959 - dense_5_acc_3: 0.0091 - dense_5_acc_4: 0.0325 - dense_5_acc_5: 0.1812 - dense_5_acc_6: 0.1956 - dense_5_acc_7:

7/7 [==============================] - 1s 149ms/step - loss: 81.7675 - dense_5_loss_1: 6.6466 - dense_5_loss_2: 5.8034 - dense_5_loss_3: 5.2855 - dense_5_loss_4: 5.0950 - dense_5_loss_5: 4.3953 - dense_5_loss_6: 4.2349 - dense_5_loss_7: 5.1142 - dense_5_loss_8: 2.7410 - dense_5_loss_9: 5.1713 - dense_5_loss_10: 4.8956 - dense_5_loss_11: 5.0033 - dense_5_loss_12: 4.1347 - dense_5_loss_13: 3.7088 - dense_5_loss_14: 3.8585 - dense_5_loss_15: 3.8817 - dense_5_loss_16: 1.7031 - dense_5_loss_17: 1.3106 - dense_5_loss_18: 0.9934 - dense_5_loss_19: 1.2977 - dense_5_loss_20: 1.0225 - dense_5_loss_21: 0.9800 - dense_5_loss_22: 0.9768 - dense_5_loss_23: 0.9803 - dense_5_loss_24: 0.8683 - dense_5_loss_25: 0.8332 - dense_5_loss_26: 0.3635 - dense_5_loss_27: 0.2736 - dense_5_loss_28: 0.0926 - dense_5_loss_29: 0.0787 - dense_5_loss_30: 0.0235 - dense_5_acc_1: 0.0272 - dense_5_acc_2: 0.1050 - dense_5_acc_3: 0.0272 - dense_5_acc_4: 0.0325 - dense_5_acc_5: 0.1812 - dense_5_acc_6: 0.1956 - dense_5_acc_7:

7/7 [==============================] - 1s 148ms/step - loss: 79.6971 - dense_5_loss_1: 6.5974 - dense_5_loss_2: 5.7330 - dense_5_loss_3: 5.2150 - dense_5_loss_4: 5.0171 - dense_5_loss_5: 4.2930 - dense_5_loss_6: 4.1177 - dense_5_loss_7: 5.0030 - dense_5_loss_8: 2.6620 - dense_5_loss_9: 5.0887 - dense_5_loss_10: 4.7890 - dense_5_loss_11: 4.9070 - dense_5_loss_12: 4.0217 - dense_5_loss_13: 3.6017 - dense_5_loss_14: 3.7638 - dense_5_loss_15: 3.7450 - dense_5_loss_16: 1.5161 - dense_5_loss_17: 1.2373 - dense_5_loss_18: 0.9408 - dense_5_loss_19: 1.2606 - dense_5_loss_20: 0.9579 - dense_5_loss_21: 0.9547 - dense_5_loss_22: 0.9457 - dense_5_loss_23: 0.9564 - dense_5_loss_24: 0.8301 - dense_5_loss_25: 0.7920 - dense_5_loss_26: 0.3188 - dense_5_loss_27: 0.2655 - dense_5_loss_28: 0.0748 - dense_5_loss_29: 0.0695 - dense_5_loss_30: 0.0215 - dense_5_acc_1: 0.0272 - dense_5_acc_2: 0.1050 - dense_5_acc_3: 0.0272 - dense_5_acc_4: 0.0000e+00 - dense_5_acc_5: 0.1994 - dense_5_acc_6: 0.2462 - dense_5_ac

In [26]:
train_model.save_weights('./weights.h5')

In [5]:
def infer_model(n_x, n_a, Tx):
    X = Input(shape=(Tx, n_x), name='input_X')
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    
    x = Lambda(lambda x: x[:, 0, :])(X)
    a = a0
    c = c0
    
    lstm_cell = LSTM(units=n_a, return_state=True)
    dense_layer = Dense(units=n_x, activation='softmax')
    
    def one_hot(x):
        x = K.argmax(x)
        x = tf.one_hot(x, n_x) 
        return x
    
    outputs = []
    
    for i in range(Tx):
        x = Lambda( 
            lambda j: tf.cond(tf.equal(tf.reduce_sum(X[:, i, :]), 0), lambda :j, lambda :X[:, i, :]))(x)
        x = Reshape(target_shape=(1, -1))(x)
        a, x, c = lstm_cell(x, initial_state=[a, c])
        x = dense_layer(a)
        outputs.append(x)
        x = Lambda(one_hot)(x)
        
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    return model

In [6]:
infer = infer_model(vocabs_size, N_A, max_len)

In [7]:
infer.load_weights('./weights.h5')

In [8]:
x = np.zeros((1, max_len, vocabs_size))
a0 = np.zeros((1, N_A))
c0 = np.zeros((1, N_A))
result = infer.predict([x, a0, c0])

In [9]:
result = np.array(result)

In [10]:
result = np.argmax(result, axis=-1)

In [11]:
result.shape

(30, 1)

In [13]:
convert_sequence_to_text(result, index2word)

'青青宰宰，，；；；；宰，，，，。。             '